<a href="https://colab.research.google.com/github/AbhiDevLab/codsoft/blob/main/Task3_FaceRecognition/face_detection_colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless numpy matplotlib scikit-learn pillow tensorflow

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import urllib.request
import pickle
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from google.colab import files
from IPython.display import display, HTML
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
from PIL import Image

print("TensorFlow version:", tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs")
    except RuntimeError as e:
        print(f"Error configuring GPU: {e}")

directories = [
    'data/known_faces',
    'data/test_images',
    'models'
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

haar_path = 'models/haarcascade_frontalface_default.xml'
if not os.path.exists(haar_path):
    print("Downloading Haar cascade model...")
    url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
    urllib.request.urlretrieve(url, haar_path)
    print(f"Downloaded to {haar_path}")

prototxt_path = 'models/deploy.prototxt'
model_path = 'models/res10_300x300_ssd_iter_140000.caffemodel'

if not os.path.exists(prototxt_path):
    print("Downloading DNN prototxt...")
    prototxt_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
    urllib.request.urlretrieve(prototxt_url, prototxt_path)
    print(f"Downloaded to {prototxt_path}")

if not os.path.exists(model_path):
    print("Downloading DNN model...")
    model_url = "https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel"
    urllib.request.urlretrieve(model_url, model_path)
    print(f"Downloaded to {model_path}")

def load_images_from_folder(folder):
    images = []
    labels = []

    for person_name in os.listdir(folder):
        person_folder = os.path.join(folder, person_name)

        if not os.path.isdir(person_folder):
            continue

        for filename in os.listdir(person_folder):
            img_path = os.path.join(person_folder, filename)

            if not os.path.isfile(img_path):
                continue

            try:
                img = cv2.imread(img_path)
                if img is not None:
                    images.append(img)
                    labels.append(person_name)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    return images, labels

def display_image(image, title=None, figsize=(10, 8)):
    plt.figure(figsize=figsize)

    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)

    if title:
        plt.title(title)

    plt.axis('off')
    plt.tight_layout()
    plt.show()

def benchmark_gpu():
    results = {}

    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            data_size = 1000
            a = tf.random.normal((data_size, data_size))
            b = tf.random.normal((data_size, data_size))

            c = tf.matmul(a, b)

            start_time = time.time()
            c = tf.matmul(a, b)
            c_val = c.numpy()
            end_time = time.time()

            for gpu in gpus:
                results[gpu.name.decode('utf-8')] = {
                    'type': 'TensorFlow',
                    'time': end_time - start_time,
                    'data_size': data_size
                }

                print(f"TensorFlow test on {gpu.name.decode('utf-8')} completed in {end_time - start_time:.4f} seconds")
        else:
            print("TensorFlow cannot access any GPUs")

            data_size = 1000
            a = np.random.rand(data_size, data_size).astype(np.float32)
            b = np.random.rand(data_size, data_size).astype(np.float32)

            start_time = time.time()
            c = np.matmul(a, b)
            end_time = time.time()

            results['CPU'] = {
                'type': 'NumPy',
                'time': end_time - start_time,
                'data_size': data_size
            }

            print(f"CPU test completed in {end_time - start_time:.4f} seconds")
    except Exception as e:
        print(f"TensorFlow benchmark failed: {e}")

    return results

class FaceDetector:
    def __init__(self, method='dnn', model_path=None):
        self.method = method

        if method == 'haar':
            if model_path is None:
                model_path = 'models/haarcascade_frontalface_default.xml'
            self.detector = cv2.CascadeClassifier(model_path)
        elif method == 'dnn':
            prototxt_path = 'models/deploy.prototxt'
            model_path = 'models/res10_300x300_ssd_iter_140000.caffemodel'

            self.detector = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

            try:
                if cv2.cuda.getCudaEnabledDeviceCount() > 0:
                    self.detector.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                    self.detector.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                    print("Using CUDA for DNN face detection")
            except:
                print("CUDA not available for OpenCV, using CPU")
        else:
            raise ValueError(f"Unsupported detection method: {method}")

    def detect_faces(self, image, min_confidence=0.5):
        if self.method == 'haar':
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.detector.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )
            return faces

        elif self.method == 'dnn':
            h, w = image.shape[:2]
            blob = cv2.dnn.blobFromImage(
                cv2.resize(image, (300, 300)),
                1.0, (300, 300),
                (104.0, 177.0, 123.0)
            )

            self.detector.setInput(blob)
            detections = self.detector.forward()

            faces = []
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]

                if confidence > min_confidence:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    faces.append((startX, startY, endX - startX, endY - startY))

            return faces

    def draw_faces(self, image, faces, color=(0, 255, 0), thickness=2):
        img_copy = image.copy()
        for (x, y, w, h) in faces:
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), color, thickness)
        return img_copy

class FaceRecognizer:
    def __init__(self, method='eigenfaces'):
        self.method = method
        self.face_size = (100, 100)

        if method == 'eigenfaces':
            self.recognizer = cv2.face.EigenFaceRecognizer_create()
        elif method == 'lbph':
            self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        elif method == 'ml':
            self.recognizer = SVC(kernel='linear', probability=True)
            self.label_encoder = LabelEncoder()
        elif method == 'deep':
            self._create_deep_model()
        else:
            raise ValueError(f"Unsupported recognition method: {method}")

    def _create_deep_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(64, activation='relu'),
        ])

        self.deep_model = model
        self.label_encoder = LabelEncoder()

    def preprocess_face(self, face):
        face = cv2.resize(face, self.face_size)

        if len(face.shape) == 3:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        face = cv2.equalizeHist(face)

        return face

    def extract_faces(self, image, face_locations):
        faces = []
        for (x, y, w, h) in face_locations:
            face = image[y:y+h, x:x+w]
            faces.append(self.preprocess_face(face))
        return faces

    def train(self, faces, labels):
        if self.method in ['eigenfaces', 'lbph']:
            self.recognizer.train(faces, np.array(labels))

        elif self.method == 'ml':
            flattened_faces = [face.flatten() for face in faces]
            encoded_labels = self.label_encoder.fit_transform(labels)
            self.recognizer.fit(flattened_faces, encoded_labels)

        elif self.method == 'deep':
            encoded_labels = self.label_encoder.fit_transform(labels)
            num_classes = len(self.label_encoder.classes_)

            if len(self.deep_model.layers) == 10:
                self.deep_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

            self.deep_model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            faces_array = np.array([face.reshape(self.face_size[0], self.face_size[1], 1) for face in faces])

            faces_array = faces_array / 255.0

            self.deep_model.fit(
                faces_array,
                encoded_labels,
                epochs=10,
                batch_size=32,
                validation_split=0.2
            )

    def predict(self, face):
        face = self.preprocess_face(face)

        if self.method in ['eigenfaces', 'lbph']:
            label, confidence = self.recognizer.predict(face)
            return label, confidence

        elif self.method == 'ml':
            face_flattened = face.flatten().reshape(1, -1)
            label = self.recognizer.predict(face_flattened)[0]
            proba = self.recognizer.predict_proba(face_flattened)[0]
            confidence = proba[label] * 100

            original_label = self.label_encoder.inverse_transform([label])[0]
            return original_label, confidence

        elif self.method == 'deep':
            face_array = face.reshape(1, self.face_size[0], self.face_size[1], 1) / 255.0

            predictions = self.deep_model.predict(face_array)
            label_index = np.argmax(predictions[0])
            confidence = predictions[0][label_index] * 100

            original_label = self.label_encoder.inverse_transform([label_index])[0]
            return original_label, confidence

    def save_model(self, path):
        if self.method in ['eigenfaces', 'lbph']:
            self.recognizer.save(path)

        elif self.method == 'ml':
            with open(path, 'wb') as f:
                pickle.dump({
                    'model': self.recognizer,
                    'encoder': self.label_encoder
                }, f)

        elif self.method == 'deep':
            model_path = path.replace('.xml', '.h5')
            self.deep_model.save(model_path)

            encoder_path = path.replace('.xml', '_encoder.pkl')
            with open(encoder_path, 'wb') as f:
                pickle.dump(self.label_encoder, f)

    def load_model(self, path):
        if self.method in ['eigenfaces', 'lbph']:
            self.recognizer.read(path)

        elif self.method == 'ml':
            with open(path, 'rb') as f:
                data = pickle.load(f)
                self.recognizer = data['model']
                self.label_encoder = data['encoder']

        elif self.method == 'deep':
            model_path = path.replace('.xml', '.h5')
            if os.path.exists(model_path):
                self.deep_model = tf.keras.models.load_model(model_path)
            else:
                raise FileNotFoundError(f"Deep learning model not found at {model_path}")

            encoder_path = path.replace('.xml', '_encoder.pkl')
            if os.path.exists(encoder_path):
                with open(encoder_path, 'rb') as f:
                    self.label_encoder = pickle.load(f)
            else:
                raise FileNotFoundError(f"Label encoder not found at {encoder_path}")

def train_face_recognition(data_dir='data/known_faces', method='deep'):
    detector = FaceDetector(method='dnn')
    recognizer = FaceRecognizer(method=method)

    print(f"Loading training images from {data_dir}...")
    images, labels = load_images_from_folder(data_dir)

    if not images:
        print("No training images found. Please add images to the data/known_faces directory.")
        return None

    print(f"Loaded {len(images)} images with {len(set(labels))} unique labels.")

    faces = []
    valid_labels = []

    start_time = time.time()
    for img, label in zip(images, labels):
        detected_faces = detector.detect_faces(img)

        if len(detected_faces) == 1:
            face_img = recognizer.extract_faces(img, detected_faces)[0]
            faces.append(face_img)
            valid_labels.append(label)

    detection_time = time.time() - start_time
    print(f"Face detection completed in {detection_time:.2f} seconds")

    if not faces:
        print("No faces detected in training images.")
        return None

    print(f"Extracted {len(faces)} faces for training.")

    print("Training face recognizer...")
    start_time = time.time()
    recognizer.train(faces, valid_labels)
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")

    model_path = f'models/face_recognizer_{method}.xml'
    recognizer.save_model(model_path)
    print(f"Model saved to {model_path}")

    return recognizer

def recognize_faces_in_image(image_path, detector, recognizer):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not load image: {image_path}")
        return

    start_time = time.time()
    faces = detector.detect_faces(image)
    detection_time = time.time() - start_time
    print(f"Detected {len(faces)} faces in {detection_time:.2f} seconds.")

    result_image = image.copy()

    start_time = time.time()
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]

        label, confidence = recognizer.predict(face)

        cv2.rectangle(result_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

        text = f"{label} ({confidence:.2f}%)"
        cv2.putText(result_image, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    recognition_time = time.time() - start_time
    print(f"Recognition completed in {recognition_time:.2f} seconds.")

    return result_image

def process_video(video_path, detector, recognizer, output_path=None):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_detection_time = 0
    total_recognition_time = 0
    start_time = time.time()

    frames = []
    max_frames = 100

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame_count += 1

        detection_start = time.time()
        faces = detector.detect_faces(frame)
        detection_time = time.time() - detection_start
        total_detection_time += detection_time

        recognition_start = time.time()
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]

            try:
                label, confidence = recognizer.predict(face)

                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                text = f"{label} ({confidence:.2f}%)"
                cv2.putText(frame, text, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            except Exception as e:
                print(f"Error recognizing face: {e}")

        recognition_time = time.time() - recognition_start
        total_recognition_time += recognition_time

        current_fps = 1.0 / (detection_time + recognition_time) if (detection_time + recognition_time) > 0 else 0
        cv2.putText(frame, f"FPS: {current_fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        if output_path:
            out.write(frame)

        if frame_count <= max_frames:
            frames.append(frame.copy())

        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames", end='\r')

    total_time = time.time() - start_time
    avg_fps = frame_count / total_time if total_time > 0 else 0
    avg_detection_time = total_detection_time / frame_count if frame_count > 0 else 0
    avg_recognition_time = total_recognition_time / frame_count if frame_count > 0 else 0

    print(f"\nPerformance Metrics:")
    print(f"Total frames processed: {frame_count}")
    print(f"Average FPS: {avg_fps:.2f}")
    print(f"Average detection time per frame: {avg_detection_time*1000:.2f} ms")
    print(f"Average recognition time per frame: {avg_recognition_time*1000:.2f} ms")

    cap.release()
    if output_path:
        out.release()

    if frames:
        print("\nPreview of processed frames:")
        preview_frames = frames[::max(1, len(frames)//5)]
        for i, frame in enumerate(preview_frames):
            print(f"Frame {i+1}/{len(preview_frames)}")
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 6))
            plt.imshow(frame_rgb)
            plt.axis('off')
            plt.show()

    return output_path if output_path else None

def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def process_webcam(detector, recognizer, num_frames=10):
    js = """
    async function captureFrame() {
        const div = document.createElement('div');
        document.body.appendChild(div);
        div.innerHTML = 'Loading webcam...';

        if (!navigator.mediaDevices || !navigator.mediaDevices.getUserMedia) {
            div.innerHTML = 'Webcam not supported in this browser';
            return null;
        }

        const video = document.createElement('video');
        video.style.display = 'none';
        div.appendChild(video);

        const canvas = document.createElement('canvas');
        canvas.style.display = 'none';
        div.appendChild(canvas);

        try {
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            video.play();
            div.innerHTML = 'Webcam active. Capturing frame...';

            await new Promise(resolve => video.onloadedmetadata = resolve);

            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            const ctx = canvas.getContext('2d');
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

            const imageData = canvas.toDataURL('image/jpeg');

            stream.getTracks().forEach(track => track.stop());

            div.remove();

            return imageData;
        } catch (error) {
            div.innerHTML = `Error accessing webcam: ${error.message}`;
            return null;
        }
    }

    captureFrame();
    """

    print("Accessing webcam...")
    print("Note: You may need to grant webcam access in your browser.")

    for i in range(num_frames):
        print(f"\nCapturing frame {i+1}/{num_frames}...")

        frame_data = eval_js(js)
        if frame_data is None:
            print("Failed to capture frame from webcam.")
            break

        frame = js_to_image(frame_data)

        faces = detector.detect_faces(frame)
        print(f"Detected {len(faces)} faces.")

        result_frame = frame.copy()
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]

            try:
                label, confidence = recognizer.predict(face)

                cv2.rectangle(result_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                text = f"{label} ({confidence:.2f}%)"
                cv2.putText(result_frame, text, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            except Exception as e:
                print(f"Error recognizing face: {e}")

        print("Processed frame:")
        result_frame_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 6))
        plt.imshow(result_frame_rgb)
        plt.axis('off')
        plt.show()

        time.sleep(3)

def upload_images():
    print("Please select one or more images to upload.")
    uploaded = files.upload()

    for filename, content in uploaded.items():
        with open(f'data/test_images/{filename}', 'wb') as f:
            f.write(content)
        print(f"Saved {filename} to data/test_images/{filename}")

    return list(uploaded.keys())

def upload_training_images(person_name):
    person_dir = f'data/known_faces/{person_name}'
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)
        print(f"Created directory: {person_dir}")

    print(f"Please select one or more images of {person_name} to upload.")
    uploaded = files.upload()

    for filename, content in uploaded.items():
        with open(f'{person_dir}/{filename}', 'wb') as f:
            f.write(content)
        print(f"Saved {filename} to {person_dir}/{filename}")

    return list(uploaded.keys())

def download_file(file_path):
    if os.path.exists(file_path):
        files.download(file_path)
        print(f"Downloaded {file_path}")
    else:
        print(f"File not found: {file_path}")

def add_person():
    person_name = input("Enter the name of the person: ")

    uploaded_files = upload_training_images(person_name)

    if uploaded_files:
        print(f"Added {len(uploaded_files)} images for {person_name}.")
        return True
    else:
        print("No images were uploaded.")
        return False

def train_model():
    method = input("Enter the recognition method (eigenfaces, lbph, ml, deep): ")
    if method not in ['eigenfaces', 'lbph', 'ml', 'deep']:
        print(f"Invalid method: {method}. Using 'deep' as default.")
        method = 'deep'

    recognizer = train_face_recognition(method=method)

    if recognizer:
        print("Training completed successfully.")
        return recognizer
    else:
        print("Training failed.")
        return None

def process_image():
    uploaded_files = upload_images()

    if not uploaded_files:
        print("No images were uploaded.")
        return

    method = input("Enter the recognition method (eigenfaces, lbph, ml, deep): ")
    if method not in ['eigenfaces', 'lbph', 'ml', 'deep']:
        print(f"Invalid method: {method}. Using 'deep' as default.")
        method = 'deep'

    detector = FaceDetector(method='dnn')
    recognizer = FaceRecognizer(method=method)

    model_path = f'models/face_recognizer_{method}.xml'
    if not os.path.exists(model_path) and method != 'deep':
        print(f"Model not found: {model_path}. Please train the model first.")
        return
    elif method == 'deep' and not os.path.exists(model_path.replace('.xml', '.h5')):
        print(f"Deep learning model not found. Please train the model first.")
        return

    recognizer.load_model(model_path)

    for filename in uploaded_files:
        image_path = f'data/test_images/{filename}'

        result = recognize_faces_in_image(image_path, detector, recognizer)

        if result is not None:
            output_path = f'data/test_images/result_{filename}'
            cv2.imwrite(output_path, result)
            print(f"Result saved to {output_path}")

            display_image(result, title=f"Face Recognition Result - {filename}")

            download = input(f"Download the result for {filename}? (y/n): ")
            if download.lower() == 'y':
                download_file(output_path)

def process_video_file():
    print("Please select a video file to upload.")
    uploaded = files.upload()

    if not uploaded:
        print("No video was uploaded.")
        return

    video_filename = list(uploaded.keys())[0]
    video_path = f'data/test_images/{video_filename}'
    with open(video_path, 'wb') as f:
        f.write(uploaded[video_filename])
    print(f"Saved {video_filename} to {video_path}")

    method = input("Enter the recognition method (eigenfaces, lbph, ml, deep): ")
    if method not in ['eigenfaces', 'lbph', 'ml', 'deep']:
        print(f"Invalid method: {method}. Using 'deep' as default.")
        method = 'deep'

    detector = FaceDetector(method='dnn')
    recognizer = FaceRecognizer(method=method)

    model_path = f'models/face_recognizer_{method}.xml'
    if not os.path.exists(model_path) and method != 'deep':
        print(f"Model not found: {model_path}. Please train the model first.")
        return
    elif method == 'deep' and not os.path.exists(model_path.replace('.xml', '.h5')):
        print(f"Deep learning model not found. Please train the model first.")
        return

    recognizer.load_model(model_path)

    output_path = f'data/test_images/result_{video_filename}'
    result = process_video(video_path, detector, recognizer, output_path)

    if result:
        print(f"Processed video saved to {output_path}")

        download = input(f"Download the processed video? (y/n): ")
        if download.lower() == 'y':
            download_file(output_path)

def use_webcam():
    method = input("Enter the recognition method (eigenfaces, lbph, ml, deep): ")
    if method not in ['eigenfaces', 'lbph', 'ml', 'deep']:
        print(f"Invalid method: {method}. Using 'deep' as default.")
        method = 'deep'

    detector = FaceDetector(method='dnn')
    recognizer = FaceRecognizer(method=method)

    model_path = f'models/face_recognizer_{method}.xml'
    if not os.path.exists(model_path) and method != 'deep':
        print(f"Model not found: {model_path}. Please train the model first.")
        return
    elif method == 'deep' and not os.path.exists(model_path.replace('.xml', '.h5')):
        print(f"Deep learning model not found. Please train the model first.")
        return

    recognizer.load_model(model_path)

    num_frames = int(input("Enter the number of frames to capture (1-10): "))
    num_frames = max(1, min(10, num_frames))

    process_webcam(detector, recognizer, num_frames)

def main_menu():
    while True:
        print("\n===== Face Detection and Recognition System =====")
        print("1. Add a person to the training dataset")
        print("2. Train the face recognition model")
        print("3. Process an image")
        print("4. Process a video")
        print("5. Use webcam")
        print("6. Check GPU performance")
        print("0. Exit")

        choice = input("\nEnter your choice (0-6): ")

        if choice == '0':
            print("Exiting...")
            break
        elif choice == '1':
            add_person()
        elif choice == '2':
            train_model()
        elif choice == '3':
            process_image()
        elif choice == '4':
            process_video_file()
        elif choice == '5':
            use_webcam()
        elif choice == '6':
            results = benchmark_gpu()
            print("\nGPU Performance Results:")
            for device, result in results.items():
                print(f"Device: {device}")
                print(f"  Type: {result['type']}")
                print(f"  Time: {result['time']:.4f} seconds")
                print(f"  Data Size: {result['data_size']}")
        else:
            print("Invalid choice. Please try again.")

main_menu()